In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Backend and import

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
kerasBKED = os.environ["KERAS_BACKEND"] 
print(kerasBKED)


tensorflow


In [ ]:
# import keras
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
from sklearn.neighbors  import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.neural_network import MLPClassifier
import pickle
import numpy as np

In [ ]:
# !pip install tensorflow==2.4.0

## **LSTM + KNN**

In [ ]:
import os

instances=500 #No of images
dirName = '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/'
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(dirName):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]
listOfFiles
# 'pneumonia_viral','pneumonia_bacterial','normal','covid'
import random
random.shuffle(listOfFiles)
len(listOfFiles)
listOfFiles=listOfFiles[0:instances]
print(listOfFiles)

['/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/normal/IM-0600-0001.jpeg', '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_viral/person422_virus_867.jpeg', '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_bacterial/person71_bacteria_349.jpeg', '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_viral/person155_virus_307.jpeg', '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/covid/ryct.2020200034.fig5-day4.jpeg', '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_bacterial/person64_bacteria_320.jpeg', '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/normal/IM-0405-0001.jpeg', '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_bacterial/person51_bacteria_247.jpeg', '/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_bacterial/person38_bacteria_195.jpeg', '/content/drive/MyDrive/fiverr_chest/co

In [ ]:
import cv2
import numpy as np
label=[]
i=0
data=np.zeros((instances,10000))
for l in listOfFiles:
  print(l)
  if '/normal/' in l:
    label.append(0)
  if '/covid/' in l:
    label.append(1)
  if '/pneumonia_bacterial/' in l:
    label.append(2)
  if 'pneumonia_viral' in l:
    label.append(3)
  
  
  from skimage import io
  im = io.imread(l, as_gray=True,)
  im=np.array(im )
  print(im.shape)
  im = cv2.resize(im, ((100, 100)))
  # flat_data_arr.append(im.flatten())
  data[i,:]=im.flatten()
  i=i+1

/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/normal/IM-0600-0001.jpeg
(1488, 1832)
/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_viral/person422_virus_867.jpeg
(211, 479)
/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_bacterial/person71_bacteria_349.jpeg
(656, 928)
/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_viral/person155_virus_307.jpeg
(1064, 1536)
/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/covid/ryct.2020200034.fig5-day4.jpeg
(1332, 1175)
/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_bacterial/person64_bacteria_320.jpeg
(614, 990)
/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/normal/IM-0405-0001.jpeg
(1619, 1952)
/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_bacterial/person51_bacteria_247.jpeg
(1018, 1322)
/content/drive/MyDrive/fiverr_chest/covid-19/four_classes/train/pneumonia_bacter

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data,label, test_size=0.33, random_state=42)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
import tensorflow
num_classes = 4
y_traincl = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_testcl = tensorflow.keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (335, 10000)
335 train samples
165 test samples


In [ ]:
#Name the feature rows as f_0, f_1, f_2...
i=0
feature_col=[]
for i in range(10000):
    feature_col.append("f_"+str(i))
    i=i+1
feature_col

['f_0',
 'f_1',
 'f_2',
 'f_3',
 'f_4',
 'f_5',
 'f_6',
 'f_7',
 'f_8',
 'f_9',
 'f_10',
 'f_11',
 'f_12',
 'f_13',
 'f_14',
 'f_15',
 'f_16',
 'f_17',
 'f_18',
 'f_19',
 'f_20',
 'f_21',
 'f_22',
 'f_23',
 'f_24',
 'f_25',
 'f_26',
 'f_27',
 'f_28',
 'f_29',
 'f_30',
 'f_31',
 'f_32',
 'f_33',
 'f_34',
 'f_35',
 'f_36',
 'f_37',
 'f_38',
 'f_39',
 'f_40',
 'f_41',
 'f_42',
 'f_43',
 'f_44',
 'f_45',
 'f_46',
 'f_47',
 'f_48',
 'f_49',
 'f_50',
 'f_51',
 'f_52',
 'f_53',
 'f_54',
 'f_55',
 'f_56',
 'f_57',
 'f_58',
 'f_59',
 'f_60',
 'f_61',
 'f_62',
 'f_63',
 'f_64',
 'f_65',
 'f_66',
 'f_67',
 'f_68',
 'f_69',
 'f_70',
 'f_71',
 'f_72',
 'f_73',
 'f_74',
 'f_75',
 'f_76',
 'f_77',
 'f_78',
 'f_79',
 'f_80',
 'f_81',
 'f_82',
 'f_83',
 'f_84',
 'f_85',
 'f_86',
 'f_87',
 'f_88',
 'f_89',
 'f_90',
 'f_91',
 'f_92',
 'f_93',
 'f_94',
 'f_95',
 'f_96',
 'f_97',
 'f_98',
 'f_99',
 'f_100',
 'f_101',
 'f_102',
 'f_103',
 'f_104',
 'f_105',
 'f_106',
 'f_107',
 'f_108',
 'f_109',
 'f_110',


# **KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support as score
model = KNeighborsClassifier(n_neighbors=3)
# Train the model using the training sets
model.fit(x_train , y_train)

#Predict Output
predicted= model.predict(x_test)  
accuracy=accuracy_score(predicted , y_test)
precision, recall, fscore, support = score(y_test, predicted)

print('Accuracy:', accuracy*100, '%.')
print('precision:  ',precision)
print('recall:  ', recall)
print('fscore: ',fscore)


Accuracy: 75.75757575757575 %.
precision:   [0.90243902 0.96666667 0.71153846 0.52380952]
recall:   [0.75510204 0.93548387 0.82222222 0.55      ]
fscore:  [0.82222222 0.95081967 0.7628866  0.53658537]


# **KNN + RF**

In [ ]:
 
clf_knn  = KNeighborsClassifier().fit(x_train , y_train)



In [ ]:
rf = RandomForestClassifier(n_estimators = 20, random_state = 42,max_features=4)
clf_rf=rf.fit(x_train , y_train)

In [ ]:
model = VotingClassifier([('ab', clf_knn), 
                          ('xb', clf_rf)], 
                          voting='soft')
model.fit(x_train , y_train)
predicted=model.predict(x_test)
accuracy=accuracy_score(predicted , y_test)
precision, recall, fscore, support = score(y_test, predicted)

print('Accuracy:', accuracy*100, '%.')
print('precision:  ',precision)
print('recall:  ', recall)
print('fscore: ',fscore)

Accuracy: 78.18181818181819 %.
precision:   [0.92857143 0.84615385 0.66666667 0.6969697 ]
recall:   [0.84782609 1.         0.73913043 0.575     ]
fscore:  [0.88636364 0.91666667 0.70103093 0.63013699]


# **DT + RF**

In [ ]:
DT = DecisionTreeClassifier(random_state=43)
model_dt = DT.fit(x_train , y_train)

In [ ]:
rf = RandomForestClassifier(n_estimators = 20, random_state = 42,max_features=4)
model_rf=rf.fit(x_train , y_train)

In [ ]:
model = VotingClassifier([('ab', model_dt), 
                          ('xb', model_rf)], 
                          voting='soft')
model.fit(x_train , y_train)
predicted=model.predict(x_test)
accuracy=accuracy_score(predicted , y_test)
precision, recall, fscore, support = score(y_test, predicted)

print('Accuracy:', accuracy*100, '%.')
print('precision:  ',precision)
print('recall:  ', recall)
print('fscore: ',fscore)

Accuracy: 64.24242424242425 %.
precision:   [0.85365854 0.775      0.56756757 0.40425532]
recall:   [0.76086957 0.93939394 0.45652174 0.475     ]
fscore:  [0.8045977  0.84931507 0.5060241  0.43678161]


# **DT + NB**

In [ ]:
DT = DecisionTreeClassifier(random_state=43)
model_dt = DT.fit(x_train , y_train)

In [ ]:
NB = GaussianNB()
model_nb = NB.fit(x_train , y_train)


In [ ]:
model = VotingClassifier([('ab', model_dt), 
                          ('xb', model_nb)], 
                          voting='soft')
model.fit(x_train , y_train)
predicted=model.predict(x_test)
accuracy=accuracy_score(predicted , y_test)
precision, recall, fscore, support = score(y_test, predicted)

print('Accuracy:', accuracy*100, '%.')
print('precision:  ',precision)
print('recall:  ', recall)
print('fscore: ',fscore)

Accuracy: 70.3030303030303 %.
precision:   [0.66666667 0.76190476 0.70588235 0.66666667]
recall:   [0.95652174 0.96969697 0.7826087  0.1       ]
fscore:  [0.78571429 0.85333333 0.74226804 0.17391304]


# **NB + LR**

In [ ]:
clf_nb = GaussianNB()
clf_nb=clf_nb.fit(x_train , y_train)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

# clf_lr = linear_model.LinearRegression().fit(x_train , y_train)
clf_lr = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(x_train , y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/gaussian_process/_gpc.py:451: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


In [ ]:
model = VotingClassifier([('ab', clf_nb), 
                          ('xb', clf_lr)], 
                          voting='soft')
model.fit(x_train , y_train)
predicted=model.predict(x_test)
accuracy=accuracy_score(predicted , y_test)
precision, recall, fscore, support = score(y_test, predicted)

print('Accuracy:', accuracy*100, '%.')
print('precision:  ',precision)
print('recall:  ', recall)
print('fscore: ',fscore)

/usr/local/lib/python3.7/dist-packages/sklearn/gaussian_process/_gpc.py:451: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Accuracy: 68.48484848484848 %.
precision:   [0.67241379 0.775      0.67272727 0.5       ]
recall:   [0.84782609 0.93939394 0.80434783 0.15      ]
fscore:  [0.75       0.84931507 0.73267327 0.23076923]


# **GPM + NB**

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0).fit(x_train , y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/gaussian_process/_gpc.py:451: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


In [ ]:
clf_nb = GaussianNB()
clf_nb=clf_nb.fit(x_train , y_train)

In [ ]:
model = VotingClassifier([('ab', clf_nb), 
                          ('xb', gpc)], 
                          voting='soft')
model.fit(x_train , y_train)
predicted=model.predict(x_test)
accuracy=accuracy_score(predicted , y_test)
precision, recall, fscore, support = score(y_test, predicted)

print('Accuracy:', accuracy*100, '%.')
print('precision:  ',precision)
print('recall:  ', recall)
print('fscore: ',fscore)

/usr/local/lib/python3.7/dist-packages/sklearn/gaussian_process/_gpc.py:451: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


Accuracy: 68.48484848484848 %.
precision:   [0.67241379 0.775      0.67272727 0.5       ]
recall:   [0.84782609 0.93939394 0.80434783 0.15      ]
fscore:  [0.75       0.84931507 0.73267327 0.23076923]
